# SEMANA 2 - MLP 1 MINIMALISTA

**Arquitetura:** Input(15) → Dense(16, ReLU) → Output(1, Sigmoid)  
**Parâmetros:** 273 (relação 3.26 amostras/parâmetro - SEGURO!)

## Objetivos desta semana:
1. ✅ Carregar dados preprocessados (Semana 1)
2. ✅ Implementar arquitetura MLP 1
3. ✅ Definir hiperparâmetros (learning rate, batch size, epochs)
4. ✅ Treinar modelo com callbacks (EarlyStopping, ModelCheckpoint)
5. ✅ Avaliar performance (ROC-AUC, loss, curvas de aprendizado)
6. ✅ Salvar modelo e métricas

## Por que esta arquitetura?
- **Simples e interpretável** - apenas 1 camada oculta
- **Baixo risco de overfitting** - 273 parâmetros para 712 amostras de treino
- **Baseline sólido** - captura padrões principais sem decorar ruídos

## 📐 ARQUITETURA VISUAL

Veja como a informação flui pela rede neural:

```
┌─────────────────────────────────────────────────────────────┐
│                    INPUT (15 features)                       │
│  [Sex, Age, Fare, HasCabin, Pclass_1, Pclass_2, ...]       │
└─────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────┐
│              CAMADA OCULTA (16 neurônios)                    │
│  [N1]  [N2]  [N3]  [N4]  ...  [N15]  [N16]                  │
│   ↓     ↓     ↓     ↓           ↓      ↓                    │
│  ReLU  ReLU  ReLU  ReLU  ...  ReLU   ReLU                   │
│                                                               │
│  ⚡ DROPOUT 0.2 (desliga 20% aleatoriamente no treino)       │
└─────────────────────────────────────────────────────────────┘
                            ↓
┌─────────────────────────────────────────────────────────────┐
│                    OUTPUT (1 neurônio)                       │
│                      Sigmoid                                 │
│                   (probabilidade 0-1)                        │
└─────────────────────────────────────────────────────────────┘
```

**Fluxo de dados:**
1. **Entrada:** Dados de 1 passageiro (15 valores entre 0-1)
2. **Camada Oculta:** Cada neurônio "olha" todas as 15 features e aprende um padrão
3. **ReLU:** Ativa o neurônio se detectou algo importante (senão = 0)
4. **Dropout:** No treino, desliga 20% dos neurônios aleatoriamente (evita decorar)
5. **Saída:** Combina os 16 neurônios → probabilidade de sobreviver (0-1)

## 🧮 CÁLCULO DE PARÂMETROS (PASSO A PASSO)

### CAMADA 1: Input (15) → Hidden (16)

**Para CADA neurônio da camada oculta:**
- 15 pesos (um conectando cada feature de entrada a este neurônio)
- 1 bias (valor de ajuste próprio do neurônio)
- **Total por neurônio:** 15 + 1 = 16 parâmetros

**Para os 16 neurônios:**
- 16 neurônios × 16 parâmetros = **256 parâmetros**

**Exemplo visual - Neurônio 1:**
```
Sex (0.7)    × peso_1  →  \
Age (0.22)   × peso_2  →   \
Fare (0.14)  × peso_3  →    \
...                    →     }→ SOMA + bias_1 → ReLU → saída_neurônio_1
Pclass_1 (1) × peso_15 →    /
                           /
                          /
```

---

### CAMADA 2: Hidden (16) → Output (1)

**Para o neurônio de saída:**
- 16 pesos (um para cada neurônio da camada oculta)
- 1 bias
- **Total:** 16 + 1 = **17 parâmetros**

---

### TOTAL GERAL
**256 + 17 = 273 parâmetros treináveis**

Com 712 amostras de treino:
- **712 / 273 = 2.61 amostras por parâmetro**
- ✅ **Margem segura!** Cada parâmetro "vê" ~3 exemplos diferentes
- ⚠️ Se fosse 0.5 amostras/param = overfitting garantido!

## 📦 Importar Bibliotecas

In [ ]:
# Bibliotecas essenciais
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.metrics import AUC, Precision, Recall

# Visualização e métricas
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report, confusion_matrix

# Configurar plot style
sns.set_style("whitegrid")

print(f"TensorFlow versão: {tf.__version__}")
print(f"Keras versão: {keras.__version__}")

## 🎲 Fixar Seeds (Reprodutibilidade)

In [ ]:
SEED = 42

# Fixar seeds
np.random.seed(SEED)
tf.random.set_seed(SEED)
import random
random.seed(SEED)

print("✅ Seeds fixadas! SEED =", SEED)

## 📂 Carregar Dados (Semana 1)

Carregamos os dados já separados e os índices salvos na Semana 1.

In [ ]:
# Carregar datasets separados
X_train = pd.read_csv('../data/processed/X_train.csv')
X_val = pd.read_csv('../data/processed/X_val.csv')
y_train = pd.read_csv('../data/processed/y_train.csv').values.ravel()
y_val = pd.read_csv('../data/processed/y_val.csv').values.ravel()

# Carregar informações do split
with open('../splits/split_indices.json', 'r') as f:
    split_info = json.load(f)

print(f"✅ Dados carregados:")
print(f"   - X_train: {X_train.shape}")
print(f"   - X_val: {X_val.shape}")
print(f"   - y_train: {y_train.shape}")
print(f"   - y_val: {y_val.shape}")
print(f"\n📊 Distribuição (treino): Classe 0={split_info['train_class_distribution']['class_0']}, Classe 1={split_info['train_class_distribution']['class_1']}")
print(f"📊 Distribuição (val): Classe 0={split_info['val_class_distribution']['class_0']}, Classe 1={split_info['val_class_distribution']['class_1']}")

## 🏗️ Definir Arquitetura MLP 1

**Estrutura:**
```
Input (15) → Dense(16, ReLU) + Dropout(0.2) + L2(0.01) → Output(1, Sigmoid)
```

**Contagem de parâmetros:**
- Camada 1: 15 × 16 + 16 (bias) = **256 parâmetros**
- Camada 2: 16 × 1 + 1 (bias) = **17 parâmetros**
- **TOTAL: 273 parâmetros**

**Relação:** 712 amostras / 273 params = **2.61 amostras por parâmetro** ✅

## 🎛️ HIPERPARÂMETROS EXPLICADOS

Antes de implementar, vamos entender **O QUE** cada hiperparâmetro faz e **POR QUÊ** escolhemos esses valores.

---

### 1️⃣ LEARNING RATE = 0.001

**O que é:** Tamanho do "passo" ao ajustar os pesos durante o treino.

**Como funciona:**
```
novo_peso = peso_antigo - (learning_rate × gradiente)
```
0,01 rapido demais - 0,001 correto. -  0,0001 mto lento, demora muito para convergir. 

**Por que 0.001?**
- ✅ Padrão comprovado para Adam optimizer
- ✅ Rápido o suficiente (converge em ~30-50 epochs)
- ✅ Estável (não fica "pulando")

---

### 2️⃣ BATCH SIZE = 32

**O que é:** Quantas amostras processar antes de atualizar os pesos.

**Comparação:**

```
BATCH = 1 (online):          BATCH = 32 (mini-batch):     BATCH = 712 (full):
Processa 1 passageiro  →     Processa 32 passageiros →    Processa TODOS →
Atualiza pesos         →     Atualiza pesos          →    Atualiza pesos
Processa próximo       →     Processa próximos 32    →    (1 atualização
(712 atualizações!)          (22 atualizações)            por época!)

✅ Rápido mas barulhento    ✅ Equilíbrio perfeito!      ⚠️ Lento, pode travar
```

**Por que 32?**
- ✅ Aprende padrões gerais (média de 32 amostras)
- ✅ Ainda rápido (22 updates por época)
- ✅ GPU-friendly (se usar aceleração)

**Analogia:** É como estudar um livro:
- Batch=1: Aprender após cada palavra (barulhento)
- Batch=32: Aprender após cada parágrafo (ideal)
- Batch=712: Ler o livro inteiro para depois estudar (lento)

---

### 3️⃣ DROPOUT = 0.2

**O que é:** Desliga 20% dos neurônios aleatoriamente durante o treino.

**Visualização:**

```
SEM DROPOUT:                   COM DROPOUT (treino):
Todas as conexões ativas       20% das conexões desligadas

N1 ──────→                     N1 ──────→
N2 ──────→                     N2  ✗✗✗✗✗  (desligado)
N3 ──────→   OUTPUT            N3 ──────→   OUTPUT
N4 ──────→                     N4 ──────→
N5 ──────→                     N5  ✗✗✗✗✗  (desligado)
...                            ...

Rede decora padrões            Rede aprende padrões
específicos (overfit!)         robustos!
```

**Por que 0.2?**
- ✅ Força a rede a aprender padrões distribuídos
- ✅ Cada neurônio não pode "depender" de outro específico
- ✅ 20% = equilíbrio (não muito, não pouco)

⚠️ **IMPORTANTE:** Dropout só funciona no **treino**! Na **predição** todos os neurônios são usados.

---

### 4️⃣ L2 REGULARIZATION (alpha = 0.01)

**O que é:** Penalidade por ter pesos grandes.

**Fórmula do Loss:**
```
Loss_total = Loss_erro + (0.01 × soma_dos_pesos²)
```

**Impacto visual:**

```
SEM L2:                        COM L2:
Peso_Age = 500.0              Peso_Age = 2.5
Peso_Sex = 300.0              Peso_Sex = 8.0

Rede decora:                   Rede generaliza:
"Se Age=0.22 EXATAMENTE,       "Idade influencia
 então sobrevive"              moderadamente"

Overfitting! ✗                 Generalização! ✓
```

**Por que 0.01?**
- ✅ Mantém pesos pequenos
- ✅ Evita overfitting
- ✅ Não é tão forte que impeça aprendizado

**Analogia:** É como uma taxa de impostos sobre pesos grandes. A rede tenta "economizar" mantendo pesos pequenos.

---

### 5️⃣ ACTIVATION = 'relu' (Rectified Linear Unit)

**O que é:** Função aplicada após a soma ponderada.

**Fórmula:**
```
ReLU(x) = max(0, x)

Se x = -2.5  →  ReLU = 0     (bloqueia)
Se x = 0.0   →  ReLU = 0     (bloqueia)
Se x = 3.7   →  ReLU = 3.7   (passa!)
```

**Por que ReLU?**
- ✅ Introduz não-linearidade (permite aprender padrões complexos)
- ✅ Simples e rápido de calcular
- ✅ Evita "vanishing gradient" (problema de outras ativações)

**Exemplo prático:**
```
Neurônio recebe: Age=0.22, Sex=1, Fare=0.14
Soma ponderada: (0.22×w1) + (1×w2) + (0.14×w3) + bias = 2.5
ReLU(2.5) = 2.5  ✓ Neurônio ATIVA (detectou padrão!)

Outro passageiro: Age=0.5, Sex=0, Fare=0.01
Soma: (0.5×w1) + (0×w2) + (0.01×w3) + bias = -1.2
ReLU(-1.2) = 0  ✗ Neurônio DESLIGA (não viu padrão relevante)
```

**Sem ReLU:** Rede seria apenas uma equação linear (inútil para padrões complexos!)

---

### 6️⃣ EARLY STOPPING (patience = 20)

**O que é:** Para o treino se validação não melhorar por 20 épocas.

**Timeline visual:**

```
ÉPOCA 1:  val_loss = 0.55  [MELHOR! ⭐ Salva modelo]
ÉPOCA 2:  val_loss = 0.52  [MELHOR! ⭐ Salva modelo]
ÉPOCA 3:  val_loss = 0.50  [MELHOR! ⭐ Salva modelo]
ÉPOCA 4:  val_loss = 0.48  [MELHOR! ⭐ Salva modelo]
ÉPOCA 5:  val_loss = 0.49  [Piorou... contador = 1]
ÉPOCA 6:  val_loss = 0.50  [Piorou... contador = 2]
...
ÉPOCA 24: val_loss = 0.51  [contador = 20 → PARA!]

Retorna ao modelo da ÉPOCA 4 (melhor val_loss = 0.48)
```

**Por que usar?**
- ✅ **Evita overfitting:** Para antes de começar a decorar
- ✅ **Economiza tempo:** Não treina 200 épocas se já convergiu
- ✅ **Pega o melhor:** Salva pesos da melhor época automaticamente

---

### 7️⃣ ADAM OPTIMIZER

**O que é:** Algoritmo que ajusta os pesos durante treino.

**Como funciona:**
1. Calcula o erro (quanto errou)
2. Calcula gradiente (direção para reduzir erro)
3. **Adapta** o learning rate automaticamente por parâmetro
4. Atualiza pesos na direção correta

**Comparação:**

```
SGD (básico):              ADAM:
Passo fixo sempre          Ajusta passo automaticamente
⚫ → ⚫ → ⚫ → ⚫ → ⚫        ⚫ ───→ ⚫ ──→ ⚫ ✓
Lento                      Rápido e eficiente
```

**Por que Adam?**
- ✅ Converge mais rápido que SGD
- ✅ Adapta learning rate sozinho
- ✅ Funciona bem na maioria dos problemas

---

**Resumo visual dos hiperparâmetros:**

| Hiperparâmetro | Valor | Impacto |
|----------------|-------|---------|
| Learning Rate | 0.001 | Velocidade de aprendizado |
| Batch Size | 32 | Balanço estabilidade/velocidade |
| Dropout | 0.2 | Anti-overfitting |
| L2 (alpha) | 0.01 | Mantém pesos pequenos |
| ReLU | - | Permite não-linearidade |
| Early Stop | 20 épocas | Para se não melhorar |
| Adam | - | Otimizador adaptativo |

## 🎯 RANDOM STATE = 42

**O que é:** Semente para gerar números aleatórios de forma **reproduzível**.

**O problema sem random state:**

```
EXECUÇÃO 1:                    EXECUÇÃO 2:
Dropout desliga neurônios:     Dropout desliga neurônios:
N2, N5, N8                     N1, N4, N7

Pesos iniciais:                Pesos iniciais:
w1 = 0.23                      w1 = -0.15
w2 = -0.41                     w2 = 0.67
...                            ...

Resultado:                     Resultado:
ROC-AUC = 0.8512               ROC-AUC = 0.8298

❌ Não dá pra reproduzir!
```

**Com random state = 42:**

```
EXECUÇÃO 1:                    EXECUÇÃO 2:
np.random.seed(42)             np.random.seed(42)
random.seed(42)                random.seed(42)
tf.random.set_seed(42)         tf.random.set_seed(42)

Dropout: N2, N5, N8            Dropout: N2, N5, N8 ✓
Pesos: w1=0.23, w2=-0.41       Pesos: w1=0.23, w2=-0.41 ✓
Resultado: 0.8512              Resultado: 0.8512 ✓

✅ IDÊNTICO! Experimento reproduzível!
```

**Onde o random state é usado:**
1. **Divisão train/val:** Quais passageiros vão para treino/validação
2. **Inicialização dos pesos:** Valores iniciais dos pesos
3. **Dropout:** Quais neurônios desligar em cada iteração
4. **Batch shuffling:** Ordem de processamento dos dados

**Por que 42?**
- Convenção da comunidade (referência "Guia do Mochileiro das Galáxias")
- Qualquer número serve, desde que seja **sempre o mesmo**!

**Importância científica:**
- ✅ Outros pesquisadores podem reproduzir seu experimento
- ✅ Você consegue debugar (mesmos resultados toda vez)
- ✅ Comparações justas entre arquiteturas

---

**Agora que entendemos TODOS os conceitos, vamos à implementação! 🚀**

In [ ]:
def criar_mlp1(input_dim=15, learning_rate=0.001):
    """
    MLP 1 - Minimalista
    Input(15) → Dense(16, ReLU) → Output(1, Sigmoid)
    """
    model = Sequential([
        Dense(16, activation='relu', 
              input_dim=input_dim,
              kernel_regularizer=l2(0.01),
              name='hidden_layer'),
        Dropout(0.2, name='dropout'),
        Dense(1, activation='sigmoid', name='output_layer')
    ])
    
    # Compilar modelo
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[AUC(name='auc'), 
                 keras.metrics.Precision(name='precision'),
                 keras.metrics.Recall(name='recall')]
    )
    
    return model

# Criar modelo
model = criar_mlp1(input_dim=X_train.shape[1])

# Resumo da arquitetura
print("="*70)
print("🏗️ ARQUITETURA MLP 1 - MINIMALISTA")
print("="*70)
model.summary()
print("="*70)

## ⚙️ Configurar Hiperparâmetros e Callbacks

**Hiperparâmetros escolhidos:**
- **Learning Rate:** 0.001 (padrão Adam, funciona bem para maioria dos casos)
- **Batch Size:** 32 (equilíbrio entre estabilidade e velocidade)
- **Epochs:** 200 (com Early Stopping para evitar overfitting)

**Callbacks:**
- **EarlyStopping:** Para se val_loss não melhorar por 20 epochs
- **ModelCheckpoint:** Salva melhor modelo (menor val_loss)
- **ReduceLROnPlateau:** Reduz learning rate se val_loss estagnar

In [ ]:
# Hiperparâmetros
BATCH_SIZE = 32
EPOCHS = 200
LEARNING_RATE = 0.001

# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='../artifacts/mlp1_best_model.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-6,
        verbose=1
    )
]

print("✅ Hiperparâmetros configurados:")
print(f"   - Batch Size: {BATCH_SIZE}")
print(f"   - Epochs: {EPOCHS}")
print(f"   - Learning Rate: {LEARNING_RATE}")
print(f"   - Callbacks: EarlyStopping, ModelCheckpoint, ReduceLROnPlateau")

## 🚀 Treinar Modelo

Iniciamos o treinamento com validação contínua para monitorar overfitting.

In [ ]:
print("🚀 Iniciando treinamento...")
print("="*70)

history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

print("="*70)
print("✅ Treinamento completo!")

## 📊 Visualizar Curvas de Aprendizado

Analisamos Loss e ROC-AUC para detectar overfitting/underfitting.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('MLP 1 - Loss por Epoch', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# ROC-AUC
axes[1].plot(history.history['auc'], label='Train AUC', linewidth=2)
axes[1].plot(history.history['val_auc'], label='Val AUC', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('ROC-AUC', fontsize=12)
axes[1].set_title('MLP 1 - ROC-AUC por Epoch', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/figures/mlp1_learning_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráficos salvos em: reports/figures/mlp1_learning_curves.png")

## 🎯 Avaliar Performance no Conjunto de Validação

In [ ]:
# Predições
y_pred_proba = model.predict(X_val)
y_pred = (y_pred_proba > 0.5).astype(int).ravel()

# Métricas
val_roc_auc = roc_auc_score(y_val, y_pred_proba)

print("="*70)
print("📊 MÉTRICAS DE VALIDAÇÃO - MLP 1")
print("="*70)
print(f"\n🎯 ROC-AUC Score: {val_roc_auc:.4f}")
print("\n" + "="*70)
print("📋 Classification Report:")
print("="*70)
print(classification_report(y_val, y_pred, target_names=['Morreu (0)', 'Sobreviveu (1)']))
print("="*70)

## 🔍 Matriz de Confusão

## 📈 Curva ROC (Receiver Operating Characteristic)

A curva ROC mostra o trade-off entre Taxa de Verdadeiros Positivos (sensibilidade) e Taxa de Falsos Positivos para diferentes thresholds de decisão. Quanto maior a área sob a curva (AUC), melhor o modelo!

In [ ]:
# Calcular curva ROC
fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plotar curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Chance (AUC = 0.50)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (FPR)', fontsize=12)
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)', fontsize=12)
plt.title('MLP 1 - Curva ROC (Validação)', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../reports/figures/mlp1_roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Curva ROC salva em: reports/figures/mlp1_roc_curve.png")
print(f"📊 ROC-AUC Score: {roc_auc:.4f}")

In [ ]:
cm = confusion_matrix(y_val, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Morreu (0)', 'Sobreviveu (1)'],
            yticklabels=['Morreu (0)', 'Sobreviveu (1)'],
            cbar_kws={'label': 'Quantidade'})
plt.xlabel('Predição', fontsize=12)
plt.ylabel('Real', fontsize=12)
plt.title('MLP 1 - Matriz de Confusão (Validação)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../reports/figures/mlp1_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Matriz de confusão salva em: reports/figures/mlp1_confusion_matrix.png")

## 💾 Salvar Métricas e Modelo Final

In [ ]:
# Salvar histórico de treino
history_dict = {
    'loss': [float(x) for x in history.history['loss']],
    'val_loss': [float(x) for x in history.history['val_loss']],
    'auc': [float(x) for x in history.history['auc']],
    'val_auc': [float(x) for x in history.history['val_auc']],
    'precision': [float(x) for x in history.history['precision']],
    'val_precision': [float(x) for x in history.history['val_precision']],
    'recall': [float(x) for x in history.history['recall']],
    'val_recall': [float(x) for x in history.history['val_recall']]
}

with open('../reports/mlp1_history.json', 'w') as f:
    json.dump(history_dict, f, indent=4)

# Salvar métricas finais
metrics = {
    'model_name': 'MLP1_Minimalista',
    'architecture': 'Input(15) -> Dense(16, ReLU) -> Output(1, Sigmoid)',
    'total_params': 273,
    'trainable_params': 273,
    'hyperparameters': {
        'batch_size': BATCH_SIZE,
        'epochs_run': len(history.history['loss']),
        'learning_rate': LEARNING_RATE,
        'dropout': 0.2,
        'l2_regularization': 0.01
    },
    'val_metrics': {
        'roc_auc': float(val_roc_auc),
        'final_val_loss': float(history.history['val_loss'][-1]),
        'best_val_loss': float(min(history.history['val_loss'])),
        'best_val_auc': float(max(history.history['val_auc']))
    },
    'confusion_matrix': cm.tolist()
}

with open('../reports/mlp1_metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

# Salvar modelo final
model.save('../artifacts/mlp1_final_model.keras')

print("="*70)
print("💾 ARQUIVOS SALVOS")
print("="*70)
print("✅ Modelo: artifacts/mlp1_best_model.keras (melhor val_loss)")
print("✅ Modelo: artifacts/mlp1_final_model.keras (final)")
print("✅ Histórico: reports/mlp1_history.json")
print("✅ Métricas: reports/mlp1_metrics.json")
print("✅ Gráficos: reports/figures/mlp1_*.png")
print("="*70)

## 🎉 RESUMO FINAL - MLP 1

In [ ]:
print("\n" + "="*70)
print("🎉 SEMANA 2 COMPLETA - MLP 1 MINIMALISTA")
print("="*70)
print(f"""
✅ Arquitetura implementada: Input(15) → Dense(16) → Output(1)
✅ Total de parâmetros: 273
✅ Relação amostras/parâmetros: {712/273:.2f}

📊 RESULTADOS:
   - ROC-AUC Validação: {val_roc_auc:.4f}
   - Melhor Val Loss: {min(history.history['val_loss']):.4f}
   - Melhor Val AUC: {max(history.history['val_auc']):.4f}
   - Epochs executados: {len(history.history['loss'])}

📈 ANÁLISE:
   - Baseline estabelecido com sucesso
   - Modelo simples e interpretável
   - Baixo risco de overfitting confirmado

🎯 PRÓXIMOS PASSOS:
   - Semana 3: Implementar MLP 2 (Moderada) ou Ensemble 3
   - Comparar performance entre arquiteturas
   - Decidir melhor modelo para produção
""")
print("="*70)